# Term Project: Detect Stop Signs in various conditions using Deep Learning
---
#### Name: Devson Butani (and Kim Lam, Sydney Ross)
#### ID: 000732711
#### LTU Honor Code: "I pledge that on all academic work that I submit, I will neither give nor receive unauthorized aid, nor will I present another person's work as my own."

# Setup Dependencies

Cuda and PyTorch are version matched for RTX 30-series GPUs. Latest versions are okay on Collab but don't work on PCs

In [ ]:
!py -m pip install --upgrade setuptools pip wheel --quiet
!py -m pip install nvidia-pyindex --quiet
!py -m pip install nvidia-cuda-runtime-cu11 --quiet
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet

In [ ]:
# Add Comet ML for data logging and tracking models
# %pip install comet_ml --quiet
import comet_ml
comet_ml.config.save(api_key="mQStuXAxGmHmK1vOsTRucvz76") # Insert API key from comet user account
comet_ml.init(project_name='Stop_sign_detection_using_yolov8')

In [ ]:
# Install ultralytics (YOLOv8)
# %pip install ultralytics --quiet
import ultralytics
ultralytics.checks()

Import PyTorch and verify GPU is connected correctly through CUDA

In [ ]:
# !pip3 install torch --index-url https://download.pytorch.org/whl/cu118 --quiet    
# !nvidia-smi
import torch
print(torch.cuda.is_available()) # check if PyTorch can see it
print(torch.cuda.get_device_name(0)) # confirm the device name

In [ ]:
torch.cuda.empty_cache()
torch.cuda.set_per_process_memory_fraction(0.6, 0)

# Get dataset from `Google Drive` OR `Roboflow`:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory to a drive folder of choice - Runs and models will be saved here
%cd '/content/drive/MyDrive/DL_data'
%pwd # Verify

In [ ]:
# Use this to select between Roboflow or already downloaded dataset
select = 0

if select == 1:
  ### Download from Roboflow
  !pip install roboflow
  from roboflow import Roboflow
  rf = Roboflow(api_key="x7krcZVIF3tFJqZdd6VJ")
  project = rf.workspace("yolo-ifyjn").project("stop-sign-detection-2")
  dataset = project.version(2).download("yolov8")
  data_path = f"{dataset.location}/data.yaml"
else: 
  ### Use Dataset inside a Google Drive folder
  # Find path from sidebar on the left
  # Find the .yaml file and copy path from it's options (three dots)
  # "Devson's"
  # data_path = '/content/drive/MyDrive/DL_data/Stop_sign_1500/data.yaml'
  data_path = r"c:\Users\devso\Desktop\YOLO_Train\StopSignDetection-1-4\data.yaml"
  # DS needs to be added to MyDrive. Shared folders do not work directly.

print(f"data_path = {data_path}")

# Train `Yolov8` model:

* Batch size and image size have huge impact on GPU memory usage
* Batches lower than 8 result in poor performance (less than 70% accuracy)
* Images smaller than 640 do not see the stop sign within 20ft
* Camera resolution on the vehicle is around 1536px
* Using native image resolution, only the nano model can be used at batch size 16. (sees 35ft far)
* Overfitting usually happens between 15 and 35 epochs for most YOLO models trained
* Medium model performs roughly 20% better but uses all the memory available in the ACTOR1 laptop
* Medium model takes around 45ms per prediction without displaying the image. (Nano: 4ms)
* Default YOLOv8m model trained on the COCO dataset is really good (90% acc) at detecting clear stop signs

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m.yaml') # build from untrained YAML parameters
results = model.train(data=data_path, batch=16, epochs=25, imgsz=640, device="0", cache=False, val=True) # train the model over our dataset
# results = model.val() # can specify data='' to verify the model with
# model.export() # Save Model is set to auto but can be changed if required

>Run logs can be found on CometML at 
https://www.comet.com/aeolus96/stop-sign-detection-using-yolov8/a3e5c5ce2c3f478c942fbabdc026d1ee?experiment-tab=panels&showOutliers=true&smoothing=0&xAxis=step OR `arbitrary_lamprey_5982` at https://www.comet.com/aeolus96/stop-sign-detection-using-yolov8/
* PPT attached as a guide to CometML
* This .ipynb file does not have terminal output due to the last few runs not working correctly. However, output of the last epoch and testing results are available under the output tab on Comet

# Training is complete, rest of the code is for testing bits and pieces for implementing in ROS
- - -
ROS package too big for canvas upload: https://github.com/Aeolus96/Route-StopSignDetector.git

In [ ]:
from ultralytics import YOLO
model = YOLO(r"C:\Users\devso\Desktop\YOLO_Train\n_best.pt")

In [ ]:
results = model(source=r"C:\Users\devso\Desktop\YOLO_Train\test_5.jpg")

In [ ]:
print(results[0])
img = results[0].plot()
# print(img)

In [ ]:
# boxes = results[0].boxes.cpu().numpy()
# print(box[1].xywh[0][2])
# print(box[2]*box[3])

In [ ]:
import cv2
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Use results to determine class and size
# Get the bounding boxes and class labels for all detected objects
boxes = results[0].boxes.cpu().numpy() # Only cast boxes to numpy
labels = results[0].names # Direct class labels access {0: 'stop-sign', 1: 'stop-sign-fake', 2: 'stop-sign-obstructed', 3: 'stop-sign-vandalized'}
# Iterate over all detected bounding boxes
at_least_one_stop_sign = False # Used to pass results to the next function
biggest_bounding_box = 0 # Used to pass results to the next function
for idx, box in enumerate(boxes):
    # Get the class label
    label = labels[int(box.cls)]
    # Check if this object is a stop sign
    if label == 'stop-sign' or label == 'stop-sign-obstructed' or label == 'stop-sign-vandalized':
        at_least_one_stop_sign = True
        # Find the width and height of the bounding box
        width = box.xywh[0][2]
        height = box.xywh[0][3]
        area = int(width * height)
        # Print the location and size of the stop sign
        print(f"[{label}] detected at ({box.xywh[0][0]:.1f}, {box.xywh[0][1]:.1f}) with size {area:.0f} sq_pix")
        if area > biggest_bounding_box:
                biggest_bounding_box = area

In [ ]:
print(at_least_one_stop_sign)
print(biggest_bounding_box)

In [ ]:
import cv2
import numpy as np
detection_size = 1536
def resize_image(img, size=(detection_size, detection_size)):
    h, w = img.shape[:2]
    c = img.shape[2] if len(img.shape) > 2 else 1
    if h == w:
        return cv2.resize(img, size, cv2.INTER_AREA)
    dif = h if h > w else w
    interpolation = cv2.INTER_AREA if dif > (size[0] + size[1]) // 2 else cv2.INTER_CUBIC
    x_pos = (dif - w) // 2
    y_pos = (dif - h) // 2
    if len(img.shape) == 2:  # Grayscale images
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos : y_pos + h, x_pos : x_pos + w] = img[:h, :w]
    else:  # 3-channel color images
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos : y_pos + h, x_pos : x_pos + w, :] = img[:h, :w, :]
    return cv2.resize(mask, size, interpolation)
from ultralytics import YOLO


# Use default YOLOv8m model trained on the COCO dataset to find bounding box of the stop sign
coco_model = YOLO(r"C:\Users\devso\Desktop\YOLO_Train\yolov8m.pt")
model = YOLO(r"C:\Users\devso\Desktop\YOLO_Train\n_best.pt")
coco_results = coco_model(source=r"C:\Users\devso\Desktop\YOLO_Train\test_10.jpg", # Image source
                classes=11, # only detect class name "stop sign" from COCO dataset
                # iou=0.5,
                # agnostic_nms=True,
                device="0") # Use GPU for inference
original_img = coco_results[0].orig_img
# results_img = coco_results[0].plot()
# cv2.imshow("YOLO-COCO-detection", results_img)
coco_boxes = coco_results[0].boxes.cpu().numpy() # Only cast boxes to numpy
at_least_one_stop_sign = False # Used to pass results to the next function
biggest_bounding_box = 0 # Used to pass results to the next function
for idx, box in enumerate(coco_boxes): # For every "stop sign" like object
    # Find the width and height of the bounding box
    x1 = int(box.xyxy[0][0])
    y1 = int(box.xyxy[0][1])
    x2 = int(box.xyxy[0][2])
    y2 = int(box.xyxy[0][3])
    box_img = original_img[y1:y2, x1:x2]
    box_img = resize_image(box_img)
    cv2.imshow("idx", box_img)
    cv2.waitKey(0)
    
    # Use the bounding box to determine the class of the stop sign
    results = model(source=box_img, imgsz=640, agnostic_nms=True, device="0")
    # Get the bounding boxes and class labels for all detected objects
    boxes = results[0].boxes.cpu().numpy() # Only cast boxes to numpy
    labels = results[0].names # Direct class labels access
    # {0: 'stop-sign', 1: 'stop-sign-fake', 2: 'stop-sign-obstructed', 3: 'stop-sign-vandalized'}
    for idx, box in enumerate(boxes):
        label = labels[int(box.cls)] # Get the class label
        # Check if this object is an actual stop sign
        if label == 'stop-sign' or label == 'stop-sign-obstructed' or label == 'stop-sign-vandalized':
            at_least_one_stop_sign = True
            # Find the width and height of the bounding box
            width = box.xywh[0][2]
            height = box.xywh[0][3]
            area = int(width * height)
            # Print the location and size of the stop sign
            print(f"[{label}] detected at ({box.xywh[0][0]:.1f}, {box.xywh[0][1]:.1f}) with size {area:.0f} sq_pix")
            if area > biggest_bounding_box:
                biggest_bounding_box = area


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print(label)